In [1]:
@show(Sys.BINDIR)
versioninfo()
import Pkg
# Descomente los siguientes comandos y vuelvalos a comentar una vez que los ejecute
# Pkg.add("Distances")
# Pkg.add("Plots")
# Pkg.update("JuMP")

Sys.BINDIR = "C:\\Users\\felip\\AppData\\Local\\Programs\\Julia 1.5.4\\bin"
Julia Version 1.5.4
Commit 69fcb5745b (2021-03-11 19:13 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: AMD Ryzen 5 3600XT 6-Core Processor            
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, znver2)


In [2]:
Pkg.status("JuMP")
using JuMP, Gurobi, Distances, Plots
const GUROBI_ENV = Gurobi.Env() # Abrir un solo ambiente de Gurobi, 
# Jupyter se marea si se crean demasiados modelos durante en una misma celda, esto lo evita.
include("preparacion.jl");

Status `C:\Users\felip\.julia\environments\v1.5\Project.toml`
  [4076af6c] JuMP v0.21.6
Academic license - for non-commercial use only - expires 2022-02-04


In [3]:
function model_STP(E, D, S)
    
    STS = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attributes(STS, "OutputFlag" => 0, "TimeLimit" => 60*60)
    nx, ny = size(D)

    @variable(STS, 0<=x[i in 1:nx, j in 1:ny; [i, j] in E]<=1, Int)
    @objective(STS, Min, sum(x[e[1], e[2]] * D[e[1], e[2]] for e in E))
    @constraint(STS, cortesimple[i in S], sum(x[e[1],e[2]] for e in E if i in e)>=1)
    
    return STS
end 

model_STP (generic function with 1 method)

In [4]:
function(read_stp(file_name))
    file = open(file_name)
    lines = readlines(file)
    n = length(lines)
    source = []
    target = []
    weight = []
    check = false
    nt = 0
    for i in 1:n
        l = lines[i]
        if occursin("E ", l)
            _, s, t, w  = split(l, " ")
            push!(source,parse(Int64,s))
            push!(target,parse(Int64,t))
            push!(weight,parse(Int64,w))
        elseif occursin("SECTION Terminals", l)
            check = true
        elseif check
            _, nt  = split(l, " ")
            nt = parse(Int64,nt)
            check = false 
        end
    end
    terminals = range(1, stop=nt)
    return source, target, weight, terminals
end;

In [5]:
function solve_STP(nombre_archivo)
    source, target, weight, terminals = read_stp(file_name)
    E = []
    num_nodes = maximum([maximum(source), maximum(target)])
    num_edges = length(source)
    W = zeros(num_nodes, num_nodes)
    for k in 1:num_edges
        i, j  = Int(source[k]), Int(target[k])
        push!(E, [i, j])
        W[i, j] = weight[k]
    end
    S = terminals
    STS=model_STP(E, W, S);
    N=num_nodes
    
    
    limite=250
    itera=0
    cortesagregados=0
    simplex=0
    
    while (itera<limite) 
        optimize!(STS)
        F=[e for e in E if value(STS[:x][e[1], e[2]]) ≈ 1]; 

        simplex=simplex + simplex_iterations(STS)

        U = encuentracomponente(N,1,F)
        
        K=setdiff(S,U)
        
        if (length(K) == 0)
            valor=objective_value(STS)
            return (N, STS)
            
        else
            for i in S
                C = encuentracomponente(N,i,F)
                @constraint(STS, sum(STS[:x][e[1],e[2]] for e in E if (e[1] in C && !(e[2] in C)) || (e[2] in C && !(e[1] in C)))>=1)
                cortesagregados+=1
            end
        end 
    
    itera=itera+1
    println("Iteración $itera, cortes $cortesagregados")                                                
    end
    if(itera==limite)
            @warn("Exceso de iteraciones")
    end
    return STS
end


solve_STP (generic function with 1 method)

In [6]:

function resuelveSTSconCallbacks(file_name)
    source, target, weight, terminals = read_stp(file_name)
    E = []
    num_nodes = maximum([maximum(source), maximum(target)])
    num_edges = length(source)
    W = zeros(num_nodes, num_nodes)
    for k in 1:num_edges
        i, j  = Int(source[k]), Int(target[k])
        push!(E, [i, j])
        W[i, j] = weight[k]
    end
    S = terminals
    STS=model_STP(E, W, S);
    N=num_nodes
    
    # Implementar función callback
    cortesagregados=0
    callbacks=0
    
    function mi_callback(cb_data)
        callbacks+=1
        # Recupera valores intermedios de cb_data y STS[:x]
        z_val=zeros(N,N)
        for e in E
            z_val[e[1], e[2]] = callback_value(cb_data, STS[:x][e[1], e[2]])
        end  
        # Revisar fraccionalidad del vector de valores encontrado, si no es integral, return nothing
        tolerancia=0.001
        if (!all( u-> abs(u-round(u))<tolerancia, z_val))
          display(string("callback en solución fraccional"))
          return N, STS
        end
        
        # Encontrar vector indicatriz F asociado
        F = [e for e in E if z_val[e[1], e[2]] ≈ 1];     
        
        #  encontrar componente U que tiene a 1. 
        U = encuentracomponente(N,1,F)
        K = setdiff(S,U)
        
        if (length(K) == 0) # si la componente toca a todos los terminales, salir de la funcion
            return N, STS
        else   # la componente no toca todos los terminales, contruir restricciones nuevas, una por cada componente
            for v in S
                C = encuentracomponente(N,v,F) #para cada componente 
                restriccion_nueva = @build_constraint(sum(STS[:x][e[1],e[2]] for e in E if (e[1] in C && !(e[2] in C)) || (e[2] in C && !(e[1] in C)))>=1)
                MOI.submit(STS, MOI.LazyConstraint(cb_data), restriccion_nueva)
                cortesagregados+=1
            end
        end
        println("callbacks $callbacks, cortes $cortesagregados")
    end  
    
    # Agregar función al modelo STS
    MOI.set(STS, MOI.LazyConstraintCallback(), mi_callback)
    # Optimizar, reportar solución y dibujar.
    optimize!(STS)
    valorSTS = objective_value(STS);
    F = [[i,j] for i in 1:N, j in 1:N if haskey(STS[:x], (i, j)) && value(STS[:x][i,j]) ≈ 1];     
    U = encuentracomponente(N,1,F)
    display(string("Árbol óptimo encontrado, de largo: ", valorSTS)) 
    display(string("Número de pasos simplex:", Int(simplex_iterations(STS))))
    display(string("Número de callbacks:", callbacks))
    display(string("Número de cortes agregados:", cortesagregados))
    display(string("Vértices de Steiner usados: ", setdiff(U,S)))
    
    return N, STS
end    

resuelveSTSconCallbacks (generic function with 1 method)

In [7]:
stp_names = ["i080-001.stp", "i080-011.stp", "i080-021.stp", "i080-031.stp", "i080-041.stp", "i080-101.stp",
             "i080-111.stp", "i080-121.stp", "i080-131.stp", "i080-141.stp", "i080-201.stp", "i080-211.stp",
             "i080-221.stp", "i080-231.stp", "i080-241.stp", "i080-301.stp", "i080-311.stp", "i080-321.stp",
             "i080-331.stp", "i080-341.stp"];

In [12]:
for file_name in stp_names
    STP=0
    start = time()
    N, STP = resuelveSTSconCallbacks("../I080/"*file_name);
    solve_time_STP = time()-start
    objective_value_STP = objective_value(STP);
    solution =[[i,j] for i in 1:N, j in 1:N if haskey(STP[:x], (i, j)) && value(STP[:x][i,j]) ≈ 1];
    txt_string = "TIME $solve_time_STP"*"\n"
    txt_string *="VALUE $objective_value_STP"*"\n"
    for e in solution
        txt_string *= "$e"*"\n"
    end
    open("solutions/sol_"*file_name[1:end-4]*".txt", "w") do file
        write(file, txt_string)
    end
    @show(file_name)
end

callbacks 1, cortes 6
callbacks 2, cortes 12
callbacks 3, cortes 18
callbacks 4, cortes 24
callbacks 5, cortes 30
callbacks 6, cortes 36
callbacks 7, cortes 42
callbacks 8, cortes 48
callbacks 9, cortes 54
callbacks 10, cortes 60
callbacks 11, cortes 66
callbacks 12, cortes 72
callbacks 13, cortes 78
callbacks 14, cortes 84
callbacks 15, cortes 90
callbacks 16, cortes 96
callbacks 17, cortes 102
callbacks 18, cortes 108
callbacks 19, cortes 114
callbacks 20, cortes 120
callbacks 21, cortes 126
callbacks 22, cortes 132
callbacks 23, cortes 138
callbacks 24, cortes 144
callbacks 25, cortes 150
callbacks 26, cortes 156
callbacks 27, cortes 162
callbacks 28, cortes 168
callbacks 29, cortes 174
callbacks 30, cortes 180
callbacks 31, cortes 186
callbacks 32, cortes 192
callbacks 33, cortes 198
callbacks 34, cortes 204
callbacks 35, cortes 210
callbacks 36, cortes 216
callbacks 37, cortes 222
callbacks 38, cortes 228
callbacks 39, cortes 234
callbacks 40, cortes 240
callbacks 41, cortes 246
c

"Árbol óptimo encontrado, de largo: 1787.0"

"Número de pasos simplex:7728"

"Número de callbacks:72"

"Número de cortes agregados:420"

...

"Árbol óptimo encontrado, de largo: 4904.0"

"Número de pasos simplex:6348073"

"Número de callbacks:2958"

"Número de cortes agregados:58960"

"Vértices de Steiner usados: [35, 79, 70, 52, 80, 33, 47, 64]"